In [ ]:
# import SharedArray as sa
# ROBOT_TYPE_NAME = 'panda'
# sa.delete("shm://{}.prepared".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.grasp_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.arm_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.rh_vals".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.result".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_in".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.response_out".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_quit".format(ROBOT_TYPE_NAME))

## set running directory

In [ ]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [ ]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir


DATA_PATH = os.path.join(os.environ['RNB_PLANNING_DIR'], "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "stowing")
try_mkdir(TEST_DATA_PATH)

In [ ]:
VISUALIZE = True
PLAY_RESULT = False
CLEARANCE = 1e-3
TOOL_RPY = (-np.pi/2,0,0)
TIMEOUT_MOTION = 5
MAX_TIME = 100


ROBOT_TYPE = RobotType.indy7

if ROBOT_TYPE in [RobotType.indy7, RobotType.indy7gripper]:
    ROBOT_NAME = "indy0"
    TOOL_LINK = "indy0_tcp"
    TOOL_NAME = "grip0"
    TOOL_XYZ = (0,0,0.14)
    HOME_POSE = (0,0,0,0,0,0)
    GRIP_DEPTH = 0.05
    
elif ROBOT_TYPE == RobotType.panda:
    ROBOT_NAME = "panda0"
    TOOL_LINK = "panda0_hand"
    TOOL_NAME = "grip0"
    TOOL_XYZ = (0,0,0.112)
    HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)
    GRIP_DEPTH = 0.03

## init combined robot config

In [ ]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                INDY_IP)]
              , connection_list=[False])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# # deprecated: s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [ ]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


In [ ]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [ ]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=False, collision=True)

## init planning scene

In [ ]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [ ]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [ ]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                 dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

## planner

In [ ]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.pair_svm import PairSVM
mplan = MoveitPlanner(pscene)
gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)

lcheck = LatticedChecker(pscene, gcheck)
pcheck = PairSVM(pscene)

## ui

In [ ]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

# Object Classes

In [ ]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask
from pkg.planning.filtering.lattice_model.scene_building import *

##
# @class WorkPlane
# @brief working plane. target and obstacle objects are generated on this plane
class WorkPlane(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.1)
    RTH_MAX = (0.5, +np.pi/2, +0.4)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.4, 0.6, 0.05)
    DIM_MAX = (0.4, 0.6, 0.05)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.8,0.8,0.2,1)
    
    def __init__(self, gscene, name, floor_height=None, *args, **kwargs):
        assert floor_height is not None, "floor_height needed"
        if floor_height > self.RTH_MIN[2]:
            self.RTH_MIN = self.RTH_MIN[:2]+(floor_height,)
        self.H = 0.4
        ObstacleBase.__init__(self, gscene, name, *args, **kwargs)
        
    def is_overlapped_with(self, gtem):
        verts, radii = gtem.get_vertice_radius()
        verts_global = np.add(np.matmul(verts, gtem.orientation_mat.transpose()), gtem.center)
        verts_wp = np.multiply(DEFAULT_VERT_DICT[self.GTYPE], tuple(self.DIM[:2])+(self.H,))
        verts_wp_global = np.add(np.matmul(verts_wp, self.geometry.orientation_mat.transpose()), 
                                 np.add(self.geometry.center, (0,0,self.H/2)))
        return get_gjk_distance(get_point_list(verts_global), get_point_list(verts_wp_global))-radii < 1e-4
            
    
##
# @class Box
# @brief box with the top and the front side open
class Box(WorkPlane):
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.6, 0.6, 0.05)
    COLOR =  (0.8,0.8,0.2,0.5)
    H_RANGE = (0.3, 0.6)
    THICKNESS = 0.05
    def __init__(self, gscene, name, H=None, **kwargs):
        WorkPlane.__init__(self, gscene=gscene, name=name, **kwargs)
        self.H = np.random.uniform(*self.H_RANGE) if H is None else H

        ## back wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_bw", link_name="base_link", 
            dims=(self.THICKNESS, self.DIM[1], self.H), center=(self.DIM[0]/2+self.THICKNESS/2, 0,self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        ## left wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_lw", link_name="base_link", 
            dims=(self.DIM[0], self.THICKNESS, self.H), center=(0, -self.DIM[1]/2-self.THICKNESS/2, self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        ## right wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_rw", link_name="base_link", 
            dims=(self.DIM[0], self.THICKNESS, self.H), center=(0, self.DIM[1]/2+self.THICKNESS/2, self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        
##
# @class SideBox
# @brief box with a side face open
class SideBox(Box):
    RTH_MIN = (0.3, -np.pi/2, -0.1)
    RTH_MAX = (0.5, +np.pi/2, +0.4)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.4, 0.6, 0.05)
    DIM_MAX = (0.4, 0.6, 0.05)
    COLOR =  (0.2,0.2,0.8,1)
    H_RANGE = (0.4, 0.4)
    THICKNESS = 0.05
    def __init__(self, gscene, name, **kwargs):
        Box.__init__(self, gscene=gscene, name=name, **kwargs)

        ## top
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_tp", link_name="base_link", 
            dims=(self.DIM[0], self.DIM[1], self.THICKNESS), center=(0, 0, self.H+self.THICKNESS/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

##
# @class TopBox
# @brief box with the top face open
class TopBox(Box):
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.6, 0.6, 0.05)
    H_RANGE = (0.3, 0.6)
    def __init__(self, gscene, name, **kwargs):
        Box.__init__(self, gscene=gscene, name=name, **kwargs)

        ## front wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_fw", link_name="base_link", 
            dims=(self.THICKNESS, self.DIM[1], self.H), center=(-self.DIM[0]/2-self.THICKNESS/2, 0,self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))
        
##
# @class Floor
# @brief Floor - lowerbound of the workspace
class Floor(ObstacleBase):
    RTH_MIN = (0.0, 0, -0.5)
    RTH_MAX = (0.0, 0, -0.5)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (3, 3, 0.1)
    DIM_MAX = (3, 3, 0.1)
    GTYPE = GEOTYPE.BOX
    
##
# @class Pole
# @brief occasional poles that obstruct robot motion
class Pole(ObstacleBase):
    RTH_MIN = (0.3, -np.pi, 0)
    RTH_MAX = (0.3, +np.pi, 0)
    RPY_MIN = (0, 0, -np.pi/6)
    RPY_MAX = (0, 0, +np.pi/6)
    DIM_MIN = (0.1, 0.1, 4)
    DIM_MAX = (0.1, 0.1, 4)
    GTYPE = GEOTYPE.BOX
    COLOR = (0.7,0.7,0.7,0.3)
    
##
# @class PlaneObstacle
# @brief Obstacles on the workplane
class PlaneObject(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.2)
    RTH_MAX = (0.8, +np.pi/2, +0.5)
    RPY_MIN = (0, 0, -np.pi)
    RPY_MAX = (0, 0, +np.pi)
    DIM_MIN = (0.05, 0.1, 0.2)
    DIM_MAX = (0.05, 0.1, 0.2)
    GTYPE = GEOTYPE.BOX
    COLOR =  (0.2,0.8,0.2,1)
    def __init__(self, gscene, name, workplane, GRIP_DEPTH, CLEARANCE, XYZ_LOC=None, **kwargs):
        self.GRIP_DEPTH = GRIP_DEPTH
        ObstacleBase.__init__(self, gscene=gscene, name=name, **kwargs)
        verts, radii = self.geometry.get_vertice_radius()
        verts_rot = np.matmul(self.geometry.orientation_mat, verts.transpose()) ## verices with global orientaion
        verts_rot_loc = np.matmul(workplane.geometry.Toff[:3,:3].transpose(), verts_rot) ## verices with local orientaion
        max_verts = np.max(verts_rot_loc, axis=-1)
        min_verts = np.min(verts_rot_loc, axis=-1)
        if XYZ_LOC is None:
            self.XYZ_LOC = np.random.uniform(np.negative(workplane.DIM)/2-min_verts+radii,np.array(workplane.DIM)/2-max_verts-radii)
            self.XYZ_LOC[2] = workplane.DIM[2]/2 + self.DIM[2]/2 + CLEARANCE
        else:
            self.XYZ_LOC = self.XYZ_LOC
        self.XYZ = np.matmul(workplane.geometry.Toff[:3,:3], self.XYZ_LOC) + workplane.geometry.Toff[:3,3]
        self.geometry.set_offset_tf(center = self.XYZ)
        self.RTH = cart2cyl(*self.XYZ)
        gscene.update_marker(self.geometry)
           
def disperse_objects(gscene, object_class, key, Nmax, workplane_on, GRIP_DEPTH, CLEARANCE=1e-3):
    clear_class(gscene, key, Nmax)
        
    obs_list = []
    while len(obs_list)<Nmax:
        iw = len(obs_list)
        obs = object_class(gscene, "{}_{}".format(key, iw), workplane_on, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)
        remove_this = False
        for obs_pre in obs_list:
            if obs_pre.is_overlapped_with(obs.geometry):
                remove_this = True
                break
        if remove_this:
            gscene.remove(obs.geometry)
        else:
            obs_list.append(obs)
    return obs_list

In [ ]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, ROBOT_TYPE.name)
try_mkdir(ROBOT_DATA_ROOT)

DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, get_now())
# DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, '20210223-051658')
try_mkdir(DATASET_PATH)
print("")
print("DATASET_PATH: {}".format(DATASET_PATH))
print("")

## Init pybullet

In [ ]:
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src/scripts/developing/pddlstream'))
from convert_pscene import *
from plan_rnb import *
connect_notebook(use_gui=False)
urdf_pybullet_path = copy_meshes(gscene)

### sampling 1-obj

In [ ]:
checkers_all = [('None', []), 
                ('Tool', [gcheck]), 
                ('ToolReach', [gcheck, rcheck]), 
                ('Full', [gcheck, rcheck, lcheck]),
                ('Pairwise', [pcheck])]

In [ ]:
Nmax_obj = 1

SAMPLE_NUM = 10

SHOW_PERIOD = 0.01

GRASP_SAMPLE = 50

results_dict_1_shelf = defaultdict(list)
# results_dict_3_plane = {int(k): v for k, v in load_json(os.path.join(DATASET_PATH, "results_dict_3_plane.json")).items()}


gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
gtimer.tic("full_loop")
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    initial_state = pscene.initialize_state(crob.home_pose)
    pscene.set_object_state(initial_state)
    from_state = initial_state.copy(pscene)
    
    for cname, checkers in checkers_all:
        mplan.motion_filters = checkers
        pscene.set_object_state(initial_state)
        gscene.update()
        
        reset_pybullet()
        robot_body, body_names, movable_bodies = pscene_to_pybullet(
            pscene, urdf_pybullet_path, tool_name=TOOL_NAME, name_exclude_list=[ROBOT_NAME])
        print('Objects:', body_names)
        saver = WorldSaver()
        mplan.reset_log(True)
        problem = pddlstream_from_problem_rnb(pscene, robot_body, body_names=body_names, 
                                              Q_init=HOME_POSE,
                                              goal_pairs = [(obj_pscene.oname, 'gp')],
                                              movable=movable_bodies, 
                                              checkers=[checker for checker in checkers if checker.BEFORE_IK],
                                              tool_name=TOOL_NAME, tool_link_name=TOOL_LINK, 
                                              mplan=mplan, timeout=TIMEOUT_MOTION, grasp_sample=GRASP_SAMPLE)
        _, _, _, stream_map, init, goal = problem
        print('Init:', init)
        print('Goal:', goal)
        print('Streams:', str_from_object(set(stream_map)))
        with Profiler():
            with LockRenderer(lock=not True):
                gtimer.tic("plan")
                solution = solve(problem, algorithm='adaptive', 
                                 unit_costs=False, success_cost=INF, max_time=MAX_TIME)
                elapsed = gtimer.toc("plan")/1000
                saver.restore()
        print_solution(solution)
        plan, cost, evaluations = solution
        res = not any(plan is status for status in [None, False])
        move_num = len(plan) if res else 0
        plan_num = len(mplan.result_log["planning"])
        fail_num = np.sum(np.logical_not(mplan.result_log["planning"]))
        results_dict_1_shelf[cname].append([elapsed, move_num, plan_num, fail_num, res])
        print("------- Result ({}): {} s -------".format(cname, elapsed))

        if VISUALIZE:
            play_pddl_plan(pscene, pscene.actor_dict["grip0"], initial_state=initial_state, 
                           body_names=body_names, plan=plan, SHOW_PERIOD=0.1)
                
    pscene.set_object_state(initial_state)
    snames = sorted(pscene.subject_dict.keys())
    gscene.update_markers_all()
    print("============ Elapsed: {} s / ETA: {} s =============".format(*
        np.round(np.divide(gtimer.eta("full_loop", i_s+1, SAMPLE_NUM), 1000), 1)))

In [ ]:
print("Planning time / Motion count / MP trials / Failed MP / Failures / Total ")
for k,v in results_dict_3_shelf.items():
    vv = np.array(v)
    if len(vv)>0:
        print(
            "{}: {}".format(k,
                            list(
                                np.round(np.mean(vv[np.where(vv[:,-1])[0], :-1], axis=0), 2)) \
                            +[np.sum(np.logical_not(vv[:,-1])), len(vv)]))
save_json(os.path.join(DATASET_PATH, "results_dict_1_shelf.json"), results_dict_1_shelf)

```
Planning time / Motion count / MP trials / Failed MP / Failures / Total 
ToolReach: [3.8, 5.8, 4.4, 0.4, 0, 10]
None: [12.45, 6.2, 3.8, 0.1, 0, 10]
Full: [4.62, 5.4, 3.2, 0.0, 0, 10]
Pairwise: [28.91, 6.1, 3.8, 0.3, 0, 10]
Tool: [17.31, 7.0, 5.5, 0.4, 0, 10]
```

### sampling 3-obj

In [ ]:
checkers_all = [('Tool', [gcheck]), 
                ('ToolReach', [gcheck, rcheck]), 
                ('Full', [gcheck, rcheck, lcheck])]

In [ ]:
Nmax_obj = 3

SAMPLE_NUM = 10

SHOW_PERIOD = 0.01

GRASP_SAMPLE = 10

results_dict_3_shelf = defaultdict(list)
# results_dict_3_plane = {int(k): v for k, v in load_json(os.path.join(DATASET_PATH, "results_dict_3_plane.json")).items()}


gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
gtimer.tic("full_loop")
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    initial_state = pscene.initialize_state(crob.home_pose)
    pscene.set_object_state(initial_state)
    from_state = initial_state.copy(pscene)
    
    for cname, checkers in checkers_all:
        mplan.motion_filters = checkers
        pscene.set_object_state(initial_state)
        gscene.update()
        
        reset_pybullet()
        robot_body, body_names, movable_bodies = pscene_to_pybullet(
            pscene, urdf_pybullet_path, tool_name=TOOL_NAME, name_exclude_list=[ROBOT_NAME])
        print('Objects:', body_names)
        saver = WorldSaver()
        mplan.reset_log(True)
        problem = pddlstream_from_problem_rnb(pscene, robot_body, body_names=body_names, 
                                              Q_init=HOME_POSE,
                                              goal_pairs = [(obj_pscene.oname, 'gp')],
                                              movable=movable_bodies, 
                                              checkers=[checker for checker in checkers if checker.BEFORE_IK],
                                              tool_name=TOOL_NAME, tool_link_name=TOOL_LINK, 
                                              mplan=mplan, timeout=TIMEOUT_MOTION, grasp_sample=GRASP_SAMPLE)
        _, _, _, stream_map, init, goal = problem
        print('Init:', init)
        print('Goal:', goal)
        print('Streams:', str_from_object(set(stream_map)))
        with Profiler():
            with LockRenderer(lock=not True):
                gtimer.tic("plan")
                solution = solve(problem, algorithm='adaptive', 
                                 unit_costs=False, success_cost=INF, max_time=MAX_TIME)
                elapsed = gtimer.toc("plan")/1000
                saver.restore()
        print_solution(solution)
        plan, cost, evaluations = solution
        res = not any(plan is status for status in [None, False])
        move_num = len(plan) if res else 0
        plan_num = len(mplan.result_log["planning"])
        fail_num = np.sum(np.logical_not(mplan.result_log["planning"]))
        results_dict_3_shelf[cname].append([elapsed, move_num, plan_num, fail_num, res])
        print("------- Result ({}): {} s -------".format(cname, elapsed))

        if VISUALIZE and PLAY_RESULT:
            play_pddl_plan(pscene, pscene.actor_dict["grip0"], initial_state=initial_state, 
                           body_names=body_names, plan=plan, SHOW_PERIOD=0.1)
                
    pscene.set_object_state(initial_state)
    snames = sorted(pscene.subject_dict.keys())
    gscene.update_markers_all()
    print("============ Elapsed: {} s / ETA: {} s =============".format(*
        np.round(np.divide(gtimer.eta("full_loop", i_s+1, SAMPLE_NUM), 1000), 1)))

In [ ]:
print("Planning time / Motion count / MP trials / Failed MP / Failures / Total ")
for k,v in results_dict_3_shelf.items():
    vv = np.array(v)
    if len(vv)>0:
        print(
            "{}: {}".format(k,
                            list(
                                np.round(np.mean(vv[np.where(vv[:,-1])[0], :-1], axis=0), 2)) \
                            +[np.sum(np.logical_not(vv[:,-1])), len(vv)]))
save_json(os.path.join(DATASET_PATH, "results_dict_3_shelf.json"), results_dict_3_shelf)

In [ ]:
import matplotlib.pyplot as plt
for cname, checker in checkers_all:
    vv = np.array(results_dict_3_shelf[cname])
    plt.plot(vv[np.where(vv[:,-1])[0], :-1][:,0])

### sampling 3-obj pole

In [ ]:
Nmax_obj = 3

SAMPLE_NUM = 10

SHOW_PERIOD = 0.01

GRASP_SAMPLE = 50

results_dict_3_pole = defaultdict(list)
# results_dict_3_plane = {int(k): v for k, v in load_json(os.path.join(DATASET_PATH, "results_dict_3_plane.json")).items()}


gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
gtimer.tic("full_loop")
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)

    pole = Pole(gscene, "pole", RTH=[0.3, gp.RTH[1], 0])


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    initial_state = pscene.initialize_state(crob.home_pose)
    pscene.set_object_state(initial_state)
    from_state = initial_state.copy(pscene)
    
    for cname, checkers in checkers_all:
        mplan.motion_filters = checkers
        pscene.set_object_state(initial_state)
        gscene.update()
        
        reset_pybullet()
        robot_body, body_names, movable_bodies = pscene_to_pybullet(
            pscene, urdf_pybullet_path, tool_name=TOOL_NAME, name_exclude_list=[ROBOT_NAME])
        print('Objects:', body_names)
        saver = WorldSaver()
        mplan.reset_log(True)
        problem = pddlstream_from_problem_rnb(pscene, robot_body, body_names=body_names, 
                                              Q_init=HOME_POSE,
                                              goal_pairs = [(obj_pscene.oname, 'gp')],
                                              movable=movable_bodies, 
                                              checkers=[checker for checker in checkers if checker.BEFORE_IK],
                                              tool_name=TOOL_NAME, tool_link_name=TOOL_LINK, 
                                              mplan=mplan, timeout=TIMEOUT_MOTION, grasp_sample=GRASP_SAMPLE)
        _, _, _, stream_map, init, goal = problem
        print('Init:', init)
        print('Goal:', goal)
        print('Streams:', str_from_object(set(stream_map)))
        with Profiler():
            with LockRenderer(lock=not True):
                gtimer.tic("plan")
                solution = solve(problem, algorithm='adaptive', 
                                 unit_costs=False, success_cost=INF, max_time=MAX_TIME)
                elapsed = gtimer.toc("plan")/1000
                saver.restore()
        print_solution(solution)
        plan, cost, evaluations = solution
        res = not any(plan is status for status in [None, False])
        move_num = len(plan) if res else 0
        plan_num = len(mplan.result_log["planning"])
        fail_num = np.sum(np.logical_not(mplan.result_log["planning"]))
        results_dict_3_pole[cname].append([elapsed, move_num, plan_num, fail_num, res])
        print("------- Result ({}): {} s -------".format(cname, elapsed))

        if VISUALIZE:
            play_pddl_plan(pscene, pscene.actor_dict["grip0"], initial_state=initial_state, 
                           body_names=body_names, plan=plan, SHOW_PERIOD=0.1)
                
    pscene.set_object_state(initial_state)
    snames = sorted(pscene.subject_dict.keys())
    gscene.update_markers_all()
    print("============ Elapsed: {} s / ETA: {} s =============".format(*
        np.round(np.divide(gtimer.eta("full_loop", i_s+1, SAMPLE_NUM), 1000), 1)))

In [ ]:
print("Planning time / Motion count / MP trials / Failed MP / Failures / Total ")
for k,v in results_dict_3_pole.items():
    vv = np.array(v)
    if len(vv)>0:
        print(
            "{}: {}".format(k,
                            list(
                                np.round(np.mean(vv[np.where(vv[:,-1])[0], :-1], axis=0), 2)) \
                            +[np.sum(np.logical_not(vv[:,-1])), len(vv)]))
save_json(os.path.join(DATASET_PATH, "results_dict_3_pole.json"), results_dict_3_pole)

## Play plan

In [ ]:
SIMULATE = False
if (plan is None) or not has_gui():
    disconnect_notebook()
else:
    command = postprocess_plan(plan)
    if SIMULATE:
#         wait_for_user('Simulate?')
        command.control()
    else:
#         wait_for_user('Execute?')
        #command.step()
        command.refine(num_steps=10).execute(time_step=0.001)

In [ ]:
for k,v in results_dict_3_shelf.items():
    vv = np.array(v)
    if len(vv)>0:
        print("{}: {}".format(k, list(np.mean(vv[np.where(vv[:,-1])[0], :-1], axis=0))+[np.sum(np.logical_not(vv[:,-1])), len(vv)]))

In [ ]:
save_json(os.path.join(DATASET_PATH, "results_dict_3_shelf.json"), results_dict_3_shelf)

In [ ]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, ROBOT_TYPE.name)
try_mkdir(ROBOT_DATA_ROOT)

DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, get_now())
# DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, '20210223-051658')
try_mkdir(DATASET_PATH)
print("")
print("DATASET_PATH: {}".format(DATASET_PATH))
print("")

## sample with pole

In [ ]:
Nmax_obj = 3

SAMPLE_NUM = 10

SHOW_PERIOD = 0.01

results_dict_3_pole = {0:[], 1:[], 2:[], 3:[]}
# results_dict_3_plane = {int(k): v for k, v in load_json(os.path.join(DATASET_PATH, "results_dict_3_plane.json")).items()}

In [ ]:
gtimer = GlobalTimer.instance()
gtimer.reset()
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)

    pole = Pole(gscene, "pole", RTH=[0.3, gp.RTH[1], 0])

    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    mplan.update_gscene()
    initial_state = pscene.initialize_state(HOME_POSE)
    pscene.set_object_state(initial_state)
    from_state = initial_state.copy(pscene)

    ppline.set_motion_planner(mplan)
    ppline.set_task_planner(tplan)

    gtimer = GlobalTimer.instance()
    gtimer.reset()



    for checker_idx in range(1,4):
        mplan.motion_filters = checkers_all[:checker_idx]
        goal_nodes = [("gp",)+deepcopy(from_state.node)[1:]]
        mplan.reset_log(True)
        gtimer.tic("plan")
        ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=600, multiprocess=False, timeout=5)
        elapsed = gtimer.toc("plan")/1000
        schedules = ppline.find_schedules()
        res = len(schedules)>0
        if res:
            schedule = ppline.sort_schedule(schedules)[0]
            move_num = len(schedule)-1
        else:
            move_num = 0
        plan_num = len(mplan.result_log["planning"])
        fail_num = np.sum(np.logical_not(mplan.result_log["planning"]))
        results_dict_3_pole[checker_idx].append([elapsed, move_num, plan_num, fail_num, res])

In [ ]:
for k,v in results_dict_3_pole.items():
    vv = np.array(v)
    if len(vv)>0:
        print("{}: {}".format(k, list(np.mean(vv[np.where(vv[:,-1])[0], :-1], axis=0))+[np.sum(np.logical_not(vv[:,-1])), len(vv)]))

In [ ]:
save_json(os.path.join(DATASET_PATH, "results_dict_3_pole.json"), results_dict_3_pole)